# <center> Démarrage rapide de TensorFlow 2 pour les experts </center>

**Source :** https://www.tensorflow.org/tutorials/quickstart/advanced?hl=fr

## Importation 

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.11.0


## Chargement et préparation du jeu de données

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 0s 0us/step


## Regrouper et mélanger l'ensemble des données avec `tf.data`

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Construction du modèle :

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

**Description du code :**

Le code fourni est une classe MyModel qui définit un modèle de réseau de neurones convolutif en utilisant l'API Keras de TensorFlow. Le modèle est construit en définissant les couches du réseau dans la méthode `init()` et en définissant la fonction `forward pass` dans la méthode `call()`.

Le modèle commence par une couche de convolution 2D avec 32 filtres de taille 3x3 et une fonction d'activation ReLU. Cette couche permet d'extraire des caractéristiques des images en entrée. Les images sont ensuite aplatis par une couche Flatten, qui transforme l'image 2D en un vecteur 1D. Cela permet aux couches denses suivantes de traiter chaque pixel de l'image de manière individuelle.

Ensuite, il y a deux couches denses. La première a 128 neurones avec une fonction d'activation ReLU. Cette couche sert à capturer des relations non linéaires entre les caractéristiques extraites par la couche de convolution. La deuxième couche dense est une couche de sortie qui a 10 neurones correspondant à chaque classe possible. Cette couche n'a pas de fonction d'activation et est donc une couche de sortie linéaire.

La fonction `forward pass`, définie dans la méthode `call()`, prend une image en entrée x et passe l'image à travers les couches définies précédemment en séquence. L'image est d'abord passée à travers la couche de convolution, puis aplatie et enfin passée à travers les deux couches denses. La sortie du modèle est une prédiction pour chaque classe possible.

Enfin, une instance du modèle est créée en appelant simplement `MyModel()`. Cette instance peut ensuite être utilisée pour entraîner le modèle, faire des prédictions ou d'autres tâches liées à l'apprentissage automatique.

**Note sur la fonction `forward pass` :**

Cette fonction n'est pas explicitement définie dans le code fourni, mais elle est implémentée dans la méthode `call()` de la classe `MyModel`.

## Optimiseur et fonction de perte :

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Sélectionnez des métriques pour mesurer la perte et la précision du modèle. Ces métriques accumulent les valeurs sur des époques, puis impriment le résultat global.

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Fonction de l'entraînement du modèle

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

## Test du modèle

In [9]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Entraînement et résultats

In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13146920502185822, Accuracy: 96.00166320800781, Test Loss: 0.0624612495303154, Test Accuracy: 97.91999816894531
Epoch 2, Loss: 0.042922403663396835, Accuracy: 98.71833038330078, Test Loss: 0.057095084339380264, Test Accuracy: 98.25
Epoch 3, Loss: 0.02206437662243843, Accuracy: 99.30166625976562, Test Loss: 0.05220847204327583, Test Accuracy: 98.29000091552734
Epoch 4, Loss: 0.013128746300935745, Accuracy: 99.54000091552734, Test Loss: 0.05968405306339264, Test Accuracy: 98.25999450683594
Epoch 5, Loss: 0.009351237677037716, Accuracy: 99.67832946777344, Test Loss: 0.06117056682705879, Test Accuracy: 98.40999603271484
